In [1]:
using IRTools

In [2]:
IRTools.@code_ir conditional(1, 2)

LoadError: UndefVarError: conditional not defined

In [3]:
function conditional(a, b)
    c = a * b
    if a >= b
        d = a * a
        return d
    else
        e = b * b
        return e
    end
end     

conditional (generic function with 1 method)

In [4]:
ir = IRTools.@code_ir conditional(1, 2)

1: (%1, %2, %3)
  %4 = %2 * %3
  %5 = %2 >= %3
  br 3 unless %5
2:
  %6 = %2 * %2
  return %6
3:
  %7 = %3 * %3
  return %7

In [5]:
IRTools.insert!(ir, IRTools.var(4), :(push!($(GlobalRef(Main, :l)), 3)))

%8

In [6]:
function trace_executed_blocks(f, args...)
    global gradient_trace
    gradient_trace = []
    
    ir = IRTools.@code_ir f(args...)
    for m in eachmatch(r"(\d.*):.*\n  %(\d*)", string(ir)) #we're using regex here because the IRTools API somehow doesn't expose the thing we want.
        block_index, var_index = parse(Int64, m[1]), parse(Int64, m[2])
        IRTools.insert!(ir, IRTools.var(var_index), :(push!($(GlobalRef(Main, :gradient_trace)), $(block_index))))
    end
    
    Base.invokelatest(IRTools.func(ir), nothing, args...)
    
    return gradient_trace
end

trace_executed_blocks (generic function with 1 method)

In [7]:
trace_executed_blocks(conditional, 4, 5)

2-element Vector{Any}:
 1
 3

In [34]:
ir

1: (%1, %2, %3)
  %8 = push!(Main.l, 3)
  %4 = %2 * %3
  %5 = %2 >= %3
  br 2 unless %5
2:
  %6 = %2 * %2
  return %6
3: (%1, %2, %3)
  br 2 unless %5

In [10]:
traced_func = IRTools.func(ir)
traced_func(nothing, 3, 5)

LoadError: UndefVarError: l not defined

In [11]:
gradient_trace

2-element Vector{Any}:
 1
 3

In [12]:
IRTools.@code_ir traced_func(nothing, 3, 5)

1: (%1, %2, %3, %4)
  %5 = IRTools.Inner.push!(Main.l, 3)
  %6 = %3 * %4
  %7 = %3 >= %4
  br 3 unless %7
2:
  %8 = %3 * %3
  return %8
3:
  %9 = %4 * %4
  return %9

In [13]:
push!(ir.blocks, ir.blocks[1])

4-element Vector{IRTools.Inner.BasicBlock}:
 IRTools.Inner.BasicBlock(IRTools.Inner.Statement[IRTools.Inner.Statement(:(push!(Main.l, 3)), Any, 0), IRTools.Inner.Statement(:(%2 * %3), Any, 1), IRTools.Inner.Statement(:(%2 >= %3), Any, 2)], Any[%1, %2, %3], Any[Any, Any, Any], IRTools.Inner.Branch[br 3 unless %5])
 IRTools.Inner.BasicBlock(IRTools.Inner.Statement[IRTools.Inner.Statement(:(%2 * %2), Any, 3)], Any[], Any[], IRTools.Inner.Branch[return %6])
 IRTools.Inner.BasicBlock(IRTools.Inner.Statement[IRTools.Inner.Statement(:(%3 * %3), Any, 5)], Any[], Any[], IRTools.Inner.Branch[return %7])
 IRTools.Inner.BasicBlock(IRTools.Inner.Statement[IRTools.Inner.Statement(:(push!(Main.l, 3)), Any, 0), IRTools.Inner.Statement(:(%2 * %3), Any, 1), IRTools.Inner.Statement(:(%2 >= %3), Any, 2)], Any[%1, %2, %3], Any[Any, Any, Any], IRTools.Inner.Branch[br 3 unless %5])

In [14]:
IRTools.deleteblock!(ir, 3)

In [15]:
IRTools.

LoadError: syntax: incomplete: premature end of input

In [16]:
ir.blocks[1].

LoadError: syntax: incomplete: premature end of input

In [17]:
bl = ir.blocks[1]

IRTools.Inner.BasicBlock(IRTools.Inner.Statement[IRTools.Inner.Statement(:(push!(Main.l, 3)), Any, 0), IRTools.Inner.Statement(:(%2 * %3), Any, 1), IRTools.Inner.Statement(:(%2 >= %3), Any, 2)], Any[%1, %2, %3], Any[Any, Any, Any], IRTools.Inner.Branch[br 2 unless %5])

In [18]:
bl.stmts

3-element Vector{IRTools.Inner.Statement}:
 IRTools.Inner.Statement(:(push!(Main.l, 3)), Any, 0)
 IRTools.Inner.Statement(:(%2 * %3), Any, 1)
 IRTools.Inner.Statement(:(%2 >= %3), Any, 2)

In [19]:
#push!(bl.stmts, 1, IRTools.Inner.Statement(:(println("yo"))))

In [20]:
for (i,b) in Iterators.enumerate(IRTools.blocks(ir))
    statements = b.stmts
    
    push!(b.stmts, IRTools.Statement(:(println($(i)))))
    
    #nb = IRTools.BasicBlock(statements, b.args, b.argtypes, b.branches)
    
    #print(nb)
    #IRTools.pushfirst!(b.ir, :(println("yo")))
    println(b)
end

LoadError: type Block has no field stmts

In [21]:
dump(ir.blocks[1].branches)

Array{IRTools.Inner.Branch}((1,))
  1: IRTools.Inner.Branch
    condition: IRTools.Inner.Variable
      id: Int64 5
    block: Int64 2
    args: Array{Any}((0,))


In [22]:
Base.invokelatest(IRTools.func(ir), nothing, 3, 100)

LoadError: UndefVarError: l not defined

In [23]:
println(ir)

1: (%1, %2, %3)
  %8 = push!(Main.l, 3)
  %4 = %2 * %3
  %5 = %2 >= %3
  br 2 unless %5
2:
  %6 = %2 * %2
  return %6
3: (%1, %2, %3)
  br 2 unless %5


In [49]:
include("autodiff_v4.jl")

trace_executed_blocks (generic function with 1 method)

In [45]:
function conditional(a, b)
    c = a * b
    if a >= b
        return a * a
    else
        c = a * b
        return c * sin(c)
    end    
end

conditional (generic function with 1 method)

In [55]:
keys(ir)

4-element Vector{IRTools.Inner.Variable}:
 %8
 %4
 %5
 %6

In [56]:
ir

1: (%1, %2, %3)
  %8 = push!(Main.l, 3)
  %4 = %2 * %3
  %5 = %2 >= %3
  br 2 unless %5
2:
  %6 = %2 * %2
  return %6
3: (%1, %2, %3)
  br 2 unless %5

In [82]:
function conditional2(a, b)
    c = a * b * sin(b) * sin(a)
    
    if a >= b * 3
        return a * a
    elseif a >= b
        c = a * b
        return c * sin(c)
    elseif (a) > sin(b)
        return sin(b)
    else
        e = sin(sin(sin(a)) * b) * a
    end    
end     

conditional2 (generic function with 1 method)

In [79]:
using Zygote

In [80]:
Zygote.gradient(conditional2, 0, 2)

(0.0, 0.0)

In [83]:
gradient_4(conditional2, 0, 2)

Dict{Any, Any}(5 => 1, 16 => 6, 20 => 7, 12 => 4, 8 => 1, 17 => 7, 19 => 7, 6 => 1, 11 => 4, 9 => 2, 14 => 5, 7 => 1, 4 => 1, 13 => 4, 15 => 5, 21 => 7, 10 => 3, 18 => 7)Any[1, 3, 5, 7]Dict{Any, Any}(4 => 13, 6 => 16, 7 => 21, 2 => 9)
skipped 16
skipped 13
skipped 12
skipped 11
skipped 9
1: (%1, %2, %3)
  %4 = Main.sin(%3)
  %5 = Main.sin(%2)
  %6 = %2 * %3 * %4 * %5
  %7 = %3 * 3
  %8 = %2 >= %7
  br 3 unless %8
2:
  %9 = %2 * %2
  br 1 (%9)
3:
  %10 = %2 >= %3
  br 5 unless %10
4:
  %11 = %2 * %3
  %12 = Main.sin(%11)
  %13 = %11 * %12
  br 1 (%13)
5:
  %14 = Main.sin(%3)
  %15 = %2 > %14
  br 7 unless %15
6:
  %16 = Main.sin(%3)
  br 1 (%16)
7:
  %17 = Main.sin(%2)
  %18 = Main.sin(%17)
  %19 = %18 * %3
  %20 = Main.sin(%19)
  %21 = %20 * %2
  %22 = 1
  %23 = 0
  %24 = %22 * %2
  %25 = %23 + %24
  %26 = 0
  %27 = %22 * %20
  %28 = %26 + %27
  %29 = 0
  %30 = cos(%19)
  %31 = %25 * %30
  %32 = %29 + %31
  %33 = 0
  %34 = %32 * %3
  %35 = %33 + %34
  %36 = 0
  %37 = %32 * %18
  %38 = 

(0.0, 0.0)

In [5]:
trace_executed_blocks(conditional, 3, 5)

2-element Vector{Any}:
 1
 3

In [11]:
ir = IRTools.@code_ir conditional(5, 3)

1: (%1, %2, %3)
  %4 = %2 * %3
  %5 = %2 >= %3
  br 3 unless %5
2:
  %6 = %2 * %2
  return %6
3:
  %7 = %2 * %3
  %8 = Main.sin(%7)
  %9 = %7 * %8
  return %9

In [12]:
IRTools.deleteblock!(ir, 3)

In [13]:
ir

1: (%1, %2, %3)
  %4 = %2 * %3
  %5 = %2 >= %3
  br 3 unless %5
2:
  %6 = %2 * %2
  return %6

In [26]:
modified_ir = gradient_4(conditional, 5, 3)

Dict{Any, Any}(5 => 1, 4 => 1, 6 => 2, 7 => 3, 9 => 3, 8 => 3)Any[1, 2]Dict{Any, Any}(2 => 6, 3 => 9)
skipped 9
skipped 8
skipped 7
1: (%1, %2, %3)
  %4 = %2 * %3
  %5 = %2 >= %3
  br 3 unless %5
2:
  %6 = %2 * %2
  %10 = 1
  %11 = 0
  %12 = %10 * %2
  %13 = %11 + %12
  %14 = 0
  %15 = %10 * %2
  %16 = %13 + %15
  %17 = nothing
  %18 = Core.tuple(%6, %16, %17)
  return %18


1: (%1, %2, %3)
  %4 = %2 * %3
  %5 = %2 >= %3
  br 3 unless %5
2:
  %6 = %2 * %2
  %10 = 1
  %11 = 0
  %12 = %10 * %2
  %13 = %11 + %12
  %14 = 0
  %15 = %10 * %2
  %16 = %13 + %15
  %17 = nothing
  %18 = Core.tuple(%6, %16, %17)
  return %18

In [5]:
Zygote.gradient(conditional, 3, 5)

(-53.725154263626024, -32.23509255817561)

In [6]:
Zygote.gradient(conditional, 5, 3)

(10.0, nothing)

In [27]:
function primary(a, b)
    x = function(n, m)
        n + m
    end
    println(x(1, 4))
    return a*b, function(c, d)
        c * d
    end
end

primary (generic function with 1 method)

In [28]:
IRTools.@code_ir primary(3, 5)

1: (%1, %2, %3)
  %4 = %new(Main.:(var"#1#3"))
  %5 = (%4)(1, 4)
  %6 = Main.println(%5)
  %7 = %2 * %3
  %8 = %new(Main.:(var"#2#4"))
  %9 = Core.tuple(%7, %8)
  return %9

In [29]:
primary(3, 5)

5


(15, var"#2#4"())